# Mixed-Integer Nonlinear Programming (MINLP)

## What is Mixed-Integer Nonlinear Programming?
**Mixed-Integer Nonlinear Programming (MINLP)** extends **Nonlinear Programming (NLP)** by allowing **integer or binary decision variables**, similar to how **Mixed-Integer Linear Programming (MILP)** extends **Linear Programming (LP)**.

In short:
- **NLP** -> Nonlinear equations with continuous variables
- **MINLP** -> Nonlinear equations **+** integer / binary logic

This allows us to model problems that involve both:
- **Nonlinear system behavior** (e.g., exponential costs, efficiency curves)
- **Discrete decisions** (e.g., on/off, choose A or B)

---

## Why MINLP is Needed
Many real-world problems involve logical decisions that affect nonlinear behavior.  
For example:
- Turning a machine **on/off** changes nonlinear energy consumption
- Choosing a technology activates a different nonlinear cost function

To connect **logic decisions (yes/no, on/off)** with **nonlinear equations**, MINLP models often use techniques such as:
- **Big-M constraints**
- Indicator constraints
- Disjunctive formulations
